<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/objectivos_reactiva_vers%C3%A3o%20ciclo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [3]:
#nome dos ficheiros
versao_ciclo="V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb"
resumo_nome=".Resumo templates_abril24.xlsb"
nome_folha_objectivo="Objetivos reativa"

In [4]:
#fazer o upload dos ficheiros
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()

Saving V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb to V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb


Saving .Resumo templates_abril24.xlsb to .Resumo templates_abril24.xlsb


In [5]:
#ler a dataframe da 1ª versão do ciclo

ciclo=pd.read_excel(versao_ciclo, sheet_name="Dados carregados").iloc[:, :7]

In [85]:
#formatar a dataframe da 1ª versão do ciclo
ciclo2=ciclo.drop(ciclo.iloc[:, 3].name, axis=1)

indice_max=ciclo2.isna().all(axis=1).idxmax()

ciclo3=ciclo2.iloc[:indice_max, :]

ciclo3.columns=["Tipo", "Objective name", "Objective value", "Tipo 2", "Objective name 2", "Objective value 2"]

ciclo3.head()


,Tipo,Objective name,Objective value,Tipo 2,Objective name 2,Objective value 2
0,Decisores,Exclusivos,1.5,Renegociação,Exclusivos,0.5
1,Decisores,MATRIZ BASE MTR,1.5,Renegociação,Convergente_MATRIZ BASE 2LO,0.0
2,Decisores,MATRIZ BASE C2C,1.5,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...,0.0
3,Decisores,MATRIZ BASE 2LO,1.5,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...,0.0
4,Decisores,MATRIZ BASE OUT,1.5,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO,0.0


In [90]:
#dividir dataframe no ciclo em 2 - ciclo_half1 inclui "decisores" e "retenção" - ciclo_half2 inclui "renegociação"
ciclo_half1=ciclo3.iloc[:, :3]

ciclo_half2=ciclo3.iloc[:, 3:]


In [60]:
#inspeccionar "ciclo_half1"

print(ciclo_half1.shape)


#ciclo_half1.head()

(69, 3)


In [91]:
#inspeccionar "ciclo_half2""
ciclo_half2=ciclo_half2.iloc[:ciclo_half2.isna().all(axis=1).idxmax(), :]

print(ciclo_half2.shape)
ciclo_half2.head()

(58, 3)


,Tipo 2,Objective name 2,Objective value 2
0,Renegociação,Exclusivos,0.5
1,Renegociação,Convergente_MATRIZ BASE 2LO,0.0
2,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...,0.0
3,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...,0.0
4,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO,0.0


In [10]:
#ler ficheiro ".resumo templates"
resumo=pd.read_excel(resumo_nome, sheet_name=nome_folha_objectivo).iloc[:, :6]

resumo2=resumo.dropna()

print(resumo2.shape)
resumo2.head()

(122, 6)


,SFID,Type,Objective Name,Begin Date,End Date,Objective\n(Value)
0,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Qualquer_Qualquer...,45383,45412.0,0.5
1,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Convergente_Qualq...,45383,45412.0,0.0
2,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_C...,45383,45412.0,0.0
3,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_C...,45383,45412.0,0.0
4,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_Q...,45383,45412.0,0.0


In [11]:
#formatar ficheiro ".resumo templates" - substituir caracteres

caracter_subs=resumo2.iloc[:, 2].apply(lambda i: "_".join(i.split("_")[:3]) + "_").values[0]

caracter_subs
resumo2["Objective Name"]=resumo2["Objective Name"].replace(regex=caracter_subs + "*", value="")

resumo2.head()


<ipython-input-11-255666f8db5d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumo2["Objective Name"]=resumo2["Objective Name"].replace(regex=caracter_subs + "*", value="")


,SFID,Type,Objective Name,Begin Date,End Date,Objective\n(Value)
0,RETENÇÃO RESIDENCIAL,Group,Renegociação_Qualquer_Qualquer_Qualquer_EXCLUS...,45383,45412.0,0.5
1,RETENÇÃO RESIDENCIAL,Group,Renegociação_Convergente_Qualquer_Qualquer_MAT...,45383,45412.0,0.0
2,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Com CM_3P Alto Va...,45383,45412.0,0.0
3,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Com CM_3P Baixo V...,45383,45412.0,0.0
4,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Qualquer_DTH_MATR...,45383,45412.0,0.0


In [94]:
#def concatenacao(db, col, str):
  #db2=db[db[col].isna()==False]
  #db2["chave"]=db2.iloc[:, 0] + "_" + db2.iloc[:, 1]
  #chave2=db[db[col].isna()==True].iloc[:, 1]

  #db2["chave"]=pd.concat([db2["chave"], chave2], axis=0)

  #return db2["chave"]


#concatenacao(ciclo_half2, "Tipo 2", "Renegociação")
#ciclo_half2[ciclo_half2["Tipo 2"].isna()==False].apply(lambda i: i)

#ciclo_half2["Tipo 2"] + "_" + ciclo_half2["Objective name 2"]

#for i, j in resumo2.loc[resumo2["Objective Name"].str.contains("renegociação", case=False), "Objective Name"].items():



#print(ciclo_half2.head())

#resumo2.loc[resumo2["Objective Name"].str.contains("renegociação", case=False), "Objective Name"].head()


ciclo_half2[ciclo_half2.iloc[:, 0].isna()==False, "Tipo 2"] + "_" + ciclo_half2[ciclo_half2.iloc[:, 0].isna()==False, "Objective name 2"]

InvalidIndexError: (0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34     True
35     True
36     True
37     True
38     True
39     True
40     True
41     True
42     True
43     True
44     True
45     True
46     True
47     True
48     True
49     True
50     True
51     True
52     True
53     True
54     True
55     True
56    False
57    False
Name: Tipo 2, dtype: bool, 'Tipo 2')